# Workflow: Read DataPoints

This notebook demonstrates how to read values from DataPoints via the Data Fabric Client.

**Workflow Steps:**
1. Initialize the client and load connector configuration
2. Discover available DataPoints from the Knowledge Graph
3. Select a DataPoint and inspect its properties
4. Read the DataPoint value (automatically resolves the correct connector)

## Step 1: Initialize Client

In [1]:
import os
import sys
from os import environ

sys.path.append(os.path.abspath(".."))

from client.connector_client import ConnectorClient

# Initialize client with Kafka broker
client = ConnectorClient(bootstrap_servers=[environ.get("KAFKA_BROKER")])

# Load connector configuration from Knowledge Graph
await client.load_connector_config()

# Show available connectors
print("Available connectors:")
client.return_connectors()

Available connectors:


dict_keys(['milling_station', 'assembly_station', 'injection_molding'])

## Step 2: Discover DataPoints

Query the Knowledge Graph to find all available DataPoints with their labels and metadata.

In [2]:
# Namespace definitions (until namespace_resolver is integrated)
DF = "http://stephantrattnig.org/data_fabric_ontology#"
RDFS = "http://www.w3.org/2000/01/rdf-schema#"

# List all DataPoints with their labels, data types, and units
datapoints_df = await client.list_instances(
    class_uri=f"{DF}DataPoint", 
    optional_props=[f"{RDFS}label", f"{DF}hasDataType", f"{DF}hasUnit", f"{DF}dataPointIdentifier"],
    pretty=True
)
datapoints_df

,entityIri,entityLabel,entityType,label,hasDataType,hasUnit,dataPointIdentifier
0,d56cae74-6db3-4f2b-a817-f71625771f77,DataPoint: AlignmentOffset,DataPoint,DataPoint: AlignmentOffset,float,millimetre,ns=2;i=57
1,b08d7da3-057e-48a7-98f6-92842a7588f9,DataPoint: SpindleSpeed,DataPoint,DataPoint: SpindleSpeed,float,revolutions per minute,ns=2;i=56
2,3dd43d1f-77e2-4466-a022-86362c0d580d,DataPoint: ClampingForce,DataPoint,DataPoint: ClampingForce,float,newton,ns=2;i=59
3,d4eb1149-2d68-4791-94f5-643b014378ad,DataPoint: InjectionPressure,DataPoint,DataPoint: InjectionPressure,float,bar,ns=2;i=56
4,29d8c9a4-80cd-495a-a60b-91af8e292d1b,DataPoint: FeedRate,DataPoint,DataPoint: FeedRate,float,millimetre per minute,ns=2;i=58
5,e358d34d-b611-42a8-b85c-556730ec8faa,DataPoint: ScrewdriverTorque,DataPoint,DataPoint: ScrewdriverTorque,float,newton metre,ns=2;i=61
6,308e2283-3c01-4c6b-a63a-de9600fcfd4f,DataPoint: MoldTemperature,DataPoint,DataPoint: MoldTemperature,float,degree Celsius,ns=2;i=58
7,7eb48039-fdae-4f89-8112-ef51378b173e,DataPoint: ToolTemperature,DataPoint,DataPoint: ToolTemperature,float,degree Celsius,ns=2;i=60
8,cce9b4d7-01c8-4b44-b241-56a7dcd22a5c,DataPoint: CycleTime,DataPoint,DataPoint: CycleTime,float,second,ns=2;i=63
9,e2189ca8-f21e-41ea-b1d8-4902f0729134,DataPoint: ScrewSpeed,DataPoint,DataPoint: ScrewSpeed,float,revolutions per minute,ns=2;i=60


## Step 3: Select and Inspect a DataPoint

Select a DataPoint from the list and view its full properties.

In [3]:
# Select a DataPoint by index (change this to explore different datapoints)
DATAPOINT_INDEX = 0

# Build the full URI from the instance UUID
selected_uuid = datapoints_df.iloc[DATAPOINT_INDEX]["entityIri"]
selected_datapoint_uri = f"http://stephantrattnig.org/instances#{selected_uuid}"

print(f"Selected DataPoint: {datapoints_df.iloc[DATAPOINT_INDEX].get('label', selected_uuid)}")
print(f"URI: {selected_datapoint_uri}")

Selected DataPoint: DataPoint: AlignmentOffset
URI: http://stephantrattnig.org/instances#d56cae74-6db3-4f2b-a817-f71625771f77


In [4]:
# Get all properties of the selected DataPoint
properties_df = await client.get_properties(
    subject_uri=selected_datapoint_uri,
    pretty=True
)
properties_df

,subjectIri,subjectType,predicate,object,subjectLabel,objectLabel,objectType
0,d56cae74-6db3-4f2b-a817-f71625771f77,DataPoint,type,DataPoint,DataPoint: AlignmentOffset,Data Point,Class
1,d56cae74-6db3-4f2b-a817-f71625771f77,DataPoint,label,DataPoint: AlignmentOffset,DataPoint: AlignmentOffset,NaN,NaN
2,d56cae74-6db3-4f2b-a817-f71625771f77,DataPoint,allowsCapability,ReadCapability,DataPoint: AlignmentOffset,Read Capability,Capability
3,d56cae74-6db3-4f2b-a817-f71625771f77,DataPoint,allowsCapability,SubscribeCapability,DataPoint: AlignmentOffset,Subscribe Capability,Capability
4,d56cae74-6db3-4f2b-a817-f71625771f77,DataPoint,dataPointIdentifier,ns=2;i=57,DataPoint: AlignmentOffset,NaN,NaN
5,d56cae74-6db3-4f2b-a817-f71625771f77,DataPoint,dataPointName,AlignmentOffset,DataPoint: AlignmentOffset,NaN,NaN
6,d56cae74-6db3-4f2b-a817-f71625771f77,DataPoint,hasAccessMode,ReadCapability,DataPoint: AlignmentOffset,Read Capability,Capability
7,d56cae74-6db3-4f2b-a817-f71625771f77,DataPoint,hasAccessMode,SubscribeCapability,DataPoint: AlignmentOffset,Subscribe Capability,Capability
8,d56cae74-6db3-4f2b-a817-f71625771f77,DataPoint,hasDataType,float,DataPoint: AlignmentOffset,NaN,NaN
9,d56cae74-6db3-4f2b-a817-f71625771f77,DataPoint,hasMetadata,OPC UA Type: AnalogItemType,DataPoint: AlignmentOffset,NaN,NaN


## Step 4: Read DataPoint Value

Use `resolve_and_read_datapoint` to automatically:
1. Find the Device that owns this DataPoint
2. Find the Connector that manages this Device
3. Send a read request via Kafka
4. Return the current value

In [5]:
# Read the current value from the DataPoint
result = await client.resolve_and_read_datapoint(
    datapoint_uri=selected_datapoint_uri
)
result

type='OPCUAReadResponsePayload' device_origin='assembly_station' value_dict={'ns=2;i=57': 0.092}


{'status': 'success',
 'device_origin': 'assembly_station',
 'message': '✅ Read successful from assembly_station',
 'values': {'ns=2;i=57': 0.092}}

In [ ]:
/e\/e